In [ ]:
# ------------------------------------------------

import os

#source /etc/os-release && echo $PRETTY_NAME
with open('/etc/os-release') as fh:
  for line in fh:
    if line and line[0] != '#':
      os.environ[line.split('=')[0]] = line.split('=')[1]
print(f'Running {os.environ["PRETTY_NAME"]}') # on {os.environ["HOST"]} as {os.environ["USER"]}')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# ------------------------------------------------
from IPython.display import display, clear_output
import time, sys
tick = time.time()
# Install mamba without resetting the kernel alla condacolab
!wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
!bash Mambaforge-Linux-x86_64.sh -bfp /usr/local
sys.path.append('/usr/local/lib/python3.10/site-packages')
!mamba config --set auto_update_conda false
!mamba install -y -c omnia -c conda-forge openmm openmmtools openff-toolkit openmmforcefields pdbfixer
!pip install -q py3Dmol
!pip install -q rdkit
!pip install -q simtk
!pip install -q plotly
!pip install -q git+https://github.com/matteoferla/DTC-compchem-practical.git
import warnings
with warnings.catch_warnings():
  warnings.simplefilter("ignore")
  # last year used pyrosetta... the warning is distracting.
  import DTC_compchem_practical as dtc
tock = time.time()
clear_output()
from typing import Sequence, List, Set, Dict, Tuple, Optional, Union
print(f'Installation time: {tock - tick}')

# Let's look at the basics of openMM

In [ ]:
from IPython.display import clear_output, display
import copy
from pathlib import Path
import requests
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import mdtraj as md
import openmm as mm
import openmm.app as mma
import openmm.unit as mmu
# nomenclature idea copied from FEGrow:
from openff.toolkit.topology import Molecule as OFFMolecule
from openff.toolkit.topology import Topology as OFFTopology
from openmmforcefields.generators import SMIRNOFFTemplateGenerator, GAFFTemplateGenerator

In [ ]:
# First we need to read in a molecule
# We will use that from tutorial 1

import pkg_resources, io

template_block: str = pkg_resources.resource_string('DTC_compchem_practical', 'data/mac1-stripped.pdb').decode()

with open('template.pdb', 'w') as fh:
  fh.write(template_block)

pdb = mma.PDBFile('template.pdb')

# Alt way:
# pdb = PDBFile(io.StringIO(template_block))

Both PDBFile (the IO for PDB files) and Modeller (the builder) have a `.topology` and `.positions` attributes.

> What do they look like and what are they describing? (remember `dir` and `type`)
(An OpenMM Quantity like a Pint Quantity. It has a value and a unit. `_value` holds the actual numpy array)

👾👾👾

### Picking a forcefield

A page on the web says this `forcefield = mma.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')`

But we will use this as we will use implicit water: `forcefield = mma.ForceField('amber14-all.xml', 'implicit/gbn2.xml')`

> What is the difference? And why can't use vacuum? I read on Reddit water is an intersubjective construct...
👾👾👾

If you mix solvents, you have mixed solvent MD (good for guessing how hydrophobics and polars will interact). A Martini model is a coarse grain model, which —007 joke redacted as it was too terrible.

If we were to use TIP3 waterbox we would need to add water:

`modeller.addSolvent(pdb.getTopology().getUnitCellDimensions(), model='tip3p', padding=1.0*mmu.nanometers, ionicStrength=0.15*mmu.molar)`

In [ ]:
forcefield = mma.ForceField('amber14-all.xml', 'implicit/gbn2.xml')

# Create system
# This class represents a molecular system. It stores the topology, the list of forces, and the list of particles.
# But not their positions!
system: mm.System = forcefield.createSystem(pdb.topology)

print('Oh no! The system has a problem!')

> What did we do wrong? Take a guess!
👾👾👾

In [ ]:
# Spot on: we forgot to add hydrogen atoms!
modeller = mma.Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens()
forcefield = mma.ForceField('amber14-all.xml', 'implicit/gbn2.xml')
system: mm.System = forcefield.createSystem(modeller.topology)
print('This system has the following forces:')
force: mm.Force
for force in system.getForces():
  print(force.__class__.__name__)

# Create an integrator as if you were to do MD...
temperature = (25+273.15) * mmu.kelvin  # this is actual temperature for entropy not MCMC
friction_coefficient = 1 / mmu.picosecond
timestep = 2 * mmu.femtosecond
integrator: mm.Integrator = mm.LangevinIntegrator(temperature, friction_coefficient, timestep)
# Create a context
# A Context stores the complete state of a simulation.
context = mm.Context(system, integrator)

# Set the positions
context.setPositions(modeller.positions)

# Compute the energy
# A State object records a snapshot of the current state of a simulation at a point in time.
state: mm.State = context.getState(getEnergy=True)
print(f'The system has {state.getPotentialEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol in potential energy')
print(f'The system has {state.getKineticEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol in kinitic energy')


> What is this Integrator thing? (cf. http://docs.openmm.org/latest/userguide/theory/04_integrators.html)

👾👾👾

> What are these forces? (cf. http://docs.openmm.org/latest/userguide/theory/02_standard_forces.html and theory page in repo)

👾👾👾

In the above, we skipped the `Platform` object. A Platform defines an implementation of all the kernels needed to perform some calculation. But we are not going to deal with CUDA drivers & co.


> 12,000 kcal/mol is a lot of energy. What is going on?

👾👾👾

Bingo. We did not energy minimise for our forcefield so it is all over the place.
Coot and similar tools for crystallography are good but they do cause energy losses for the perfect position.

### energy minimise

`mm.LocalEnergyMinimizer.minimize(context)` is one way to do it.

The other way does not use the context but a new object

```python
# Create a Simulation object
simulation = mma.Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
```

Obviously we don't have all day, so won't and use one prepared earlier (`mac1-stripped.min.pdb`).
But do energy minimise your templates!

In [ ]:
# Get the energy of this protein!

pdb = 👾👾👾
system = 👾👾👾

In [ ]:
# Now let do a simulation of a small 10 steps
integrator: mm.Integrator = mm.LangevinIntegrator(temperature, friction_coefficient, timestep)
simulation = mma.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.step(5)

state: mm.State = similation.context.getState(getEnergy=True)
print(f'The system has {state.getPotentialEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol in potential energy')
print(f'The system has {state.getKineticEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol in kinitic energy')

# if you get 'OpenMMException: Called setPositions() on a Context with the wrong number of positions'
# it is because you skipped the about step. Please stand up and go to the corner and recide Gaudeamus x3 times

Now, let's get back to basics.

# Lenard-Jones

> We saw the Lenard-Jones potential. What does it do and look like?

👾👾👾

In [ ]:
import openmmtools as mmt
import numpy as np
import numpy.typing as npt

LJpair: mmt.testsystems.TestSystem = mmt.testsystems.LennardJonesPair()
system: mm.System = LJpair.system

print('This system has the following forces:')
force: mm.Force
for force in LJpair.system.getForces():
  print(force.__class__.__name__)

print('This system has the following atoms:')
atom: mma.topology.Atom
for atom in LJpair.topology.atoms():
  print(atom.index, atom.element.name)

# To calculate the energy we need some things set up first
# even if we arent integrating system in time
# Create an integrator (simple Langevin integrator in this example)
temperature = (25+273.15) * mmu.kelvin  # this is actual temperature for entropy not MCMC
friction_coefficient = 1 / mmu.picosecond
timestep = 2 * mmu.femtosecond
integrator: mm.Integrator = mm.LangevinIntegrator(temperature, friction_coefficient, timestep)
context = mm.Context(LJpair.system, integrator)
context.setPositions(LJpair.positions)

# Compute the energy
state: mm.State = context.getState(getEnergy=True)
print(f'The system has {state.getPotentialEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol potential energy')
print(f'The system has {state.getKineticEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol kinetic energy')

position: npt.NDArray[np.float32] = LJpair.positions._value
distance: np.float32 = np.linalg.norm(position[0,:] - position[1,:])
print(f'the distance of the two atoms ins {distance:.2f} Å')

> Why is the kinetic energy zero? Even if we _set_ an integrator in Langevin dynamics?

👾👾👾

In [ ]:
# Let's do a simple experiment to test this

from typing import Dict
scores: Dict[float, float] = {}

for i in np.arange(0., 10., 0.01):
  position[1,0] = i
  context.setPositions(LJpair.positions)  # it can accept Quantity or numpy
  state: mm.State = context.getState(getEnergy=True)
  energy: mmu.Quantity = state.getPotentialEnergy()
  scores[float(i)] = float(energy.value_in_unit(mmu.kilocalorie_per_mole))

import plotly.express as px
import pandas as pd

fig = px.scatter(pd.DataFrame(list(scores.items()), columns=['distance', 'potential']), 'distance', 'potential')
fig.update_layout(yaxis_range=[-20,20])
fig.show()

In [ ]:
# Let's get back to our protein and its hit

hit_block: str = dtc.get_data('QRU.mol')
# Hit as a rdkit object
hit: Chem.Mol = Chem.MolFromMolBlock(hit_block)

# But something is wrong!
dtc.get_mols_view(whiteCarbon=hit).show()

> What did we forget?

👾👾👾

In [ ]:
# Yes, hydrogens!

hit_h = AllChem.AddHs(hit)

dtc.get_mols_view(whiteCarbon=hit_h).show()

Ops. We did it wrong. `AllChem.AddHs` did not add coordinates.
`help(AllChem.AddHs)` will say what it is...

In [ ]:
# Hit as a OpenFF object
off_hit = OFFMolecule.from_rdkit(hit_h, allow_undefined_stereo=True)
# Now as a OpenMM Modeller one
hit_topo: mma.Topology = OFFTopology.from_molecules([off_hit]).to_openmm()
hit_pos: mmu.Quantity = off_hit.conformers[0].to_openmm()
omm_hit = mma.Modeller(hit_topo, hit_pos)
# If we hadn't added the hydrogen in RDKit `modeller.addHydrogens()` will have worked too as topology will be with bond order.
holo = mma.Modeller(pdb.topology, pdb.positions)
holo.add(omm_hit.topology, omm_hit.positions)

## What about the forcefield?
It has bond order, but how does it behave?
It needs to be parameterised for the forcefield. In Amber and co, these files were XML and you see those around.
Here we can use the Amber GAFF or Smirnoff amongst others. First let's look at GAFF2 FF.

In OpenFF we can do `gaff = GAFFTemplateGenerator(molecules=off_hit, forcefield='gaff-2.11')`, but need to do a lot of faff to get the atom types. Instead I did the following ancient line to covert the molecules
`antechamber -i hQRU.mol -fi mdl -o QRU.mol2 -fo mol2 -at gaff2 -nc -1 -rn QRU -c gas` (antechamber is the prep tool for Amber forcefields). The output file is [here](https://github.com/matteoferla/DTC-compchem-practical/blob/main/DTC_compchem_practical/data/QRU.mol2).


In the Atoms block, we have index, name, x, y, z, atom type, residue number, residue name (technically chemical component identifier) and partial charge.
NB. In molecular mechanics atom types has a different meaning that in most deep learning chemistry models, where it is used as a synonym for element symbol.

> What atom types can you see (cf. https://ambermd.org/antechamber/gaff.html#atomtype or paper for GAFF2)?

👾👾👾

> Why use atom types instead of element symbols?

👾👾👾

> Does an `aromatic sp2 C` form trans or cis isomers? What about `aliphatic sp2 C`? Is the hydrogen - heavy atom bond the same length?

👾👾👾

In [ ]:
# Having look at the classics, we will move into the modern era
smirnoff = SMIRNOFFTemplateGenerator(molecules=off_hit)
forcefield = mma.ForceField('amber14-all.xml', 'implicit/gbn2.xml')
forcefield.registerTemplateGenerator(smirnoff.generator)

# Now the molecule will be recognised and we can get the energy of the complex
system: mm.System = forcefield.createSystem(holo.topology)
integrator: mm.Integrator = mm.LangevinIntegrator(temperature, friction_coefficient, timestep)
context = mm.Context(system, integrator)
context.setPositions(holo.positions)
state: mm.State = context.getState(getEnergy=True)
print(f'The system has {state.getPotentialEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol in potential energy')
print(f'The system has {state.getKineticEnergy().value_in_unit(mmu.kilocalorie_per_mole):.1f} kcal/mol in kinitic energy')

This is just the internal energy. In reality to get Gibbs free energy we need entropy.
This requires a simulation in both bound and unbound form. See FEP or dynamic undocking.